In [1]:
from utils.audio_dataset_utils import train_val_dataset
from network_models.soundsream_models_and_utils.ss_model_dim_red import SSDimRedModel
from network_models.soundsream_models_and_utils.ss_model_conv import SSConvModel3Sec
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
from torch.utils.data import DataLoader

batch_size = 1

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings.pkl", device="cuda")


modelDimRed = SSDimRedModel().cuda()
modelConv = SSConvModel3Sec(xSize=512, ySize=175).cuda()



2023-02-09 13:58:16 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-02-09 13:58:16.495202: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 13:58:17.111351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-09 13:58:17.111408: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-09 13:58:17.111412: W tensorflow/compiler/tf

In [2]:
def generateParameters(device, model, dataset):
    loader = DataLoader(dataset, shuffle=False, batch_size=1)
    emo_angry = []
    emo_disgust = []
    emo_fear = []
    emo_happy = []
    emo_neutral = []
    emo_sad = []
    emo_surprise = []

    with torch.no_grad():
        for batch, (X, z) in enumerate(loader):
            X, z = X.to(device),  z.to(device)
            dims, pred = model(X, return_with_dims= True)
            labels = data_set.encoded_dataset.label_list[z[0].nonzero().cpu().numpy()[0][0]]

            match (labels):
                case "neutral":
                    emo_neutral += dims
                case "happy":
                    emo_happy += dims
                case "sad":
                    emo_sad += dims
                case "angry":
                    emo_angry += dims
                case "fear":
                    emo_fear += dims
                case "disgust":
                    emo_disgust += dims
                case "surprise":
                    emo_surprise += dims
            if batch % 100 == 0:
                print(f"{batch*batch_size} of {len(dataset)}")

    return emo_angry, emo_disgust, emo_fear, emo_happy, emo_neutral, emo_sad, emo_surprise

In [3]:
emo_angry, emo_disgust, emo_fear, emo_happy, emo_neutral, emo_sad, emo_surprise = generateParameters("cuda", modelDimRed, data_set)

/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0 of 6092
100 of 6092
200 of 6092
300 of 6092
400 of 6092
500 of 6092
600 of 6092
700 of 6092
800 of 6092
900 of 6092
1000 of 6092
1100 of 6092
1200 of 6092
1300 of 6092
1400 of 6092
1500 of 6092
1600 of 6092
1700 of 6092
1800 of 6092
1900 of 6092
2000 of 6092
2100 of 6092
2200 of 6092
2300 of 6092
2400 of 6092
2500 of 6092
2600 of 6092
2700 of 6092
2800 of 6092
2900 of 6092
3000 of 6092
3100 of 6092
3200 of 6092
3300 of 6092
3400 of 6092
3500 of 6092
3600 of 6092
3700 of 6092
3800 of 6092
3900 of 6092
4000 of 6092
4100 of 6092
4200 of 6092
4300 of 6092
4400 of 6092
4500 of 6092
4600 of 6092
4700 of 6092
4800 of 6092
4900 of 6092
5000 of 6092
5100 of 6092
5200 of 6092
5300 of 6092
5400 of 6092
5500 of 6092
5600 of 6092
5700 of 6092
5800 of 6092
5900 of 6092
6000 of 6092
